# Introduction to Jupyter Notebooks
This lesson will introduce the Jupyter Notebook interface. We will use the interface to run and write, yes, write, some Python code for text data analysis.

By the end of this lesson, learners should be able to:
1. Explain the difference between markdown and code blocks in Jupyter Notebooks
2. Execute pre-written Python code to analyze newspaper text
3. Modify Python code to change the settings of the analysis

## What is this Jupyter Notebook thing?

Jupyter Notebooks are effectively made up of "cells". We can start by thinking of each cell being equivalent to a paragraph on a page. There is an order in which paragraphs and cells appear, and that order matters. In Jupyter Notebooks, the cells come in two flavors and a single notebook (like the one we are working in now) with have both types of cells. 
+ The first is called "markdown", which is text, like you are reading now. We can use some syntax in the text to format the cells in particular ways. For example, we can create italic text by using the underscore symbol ("\_") at the beginning and ending of the text we want to italicize. So when we write "\_italic\_" in a markdown block, it will show up as _italic_.
+ The second kind of cell is a "code" cell, that contains computer code in a language like Python or R. This is where the fun comes in.

**Do some markdown stuff?**

In [ ]:
print("Collections as Data")

## So what is Python then?

Add brief explantion of python?

In [ ]:
print("Hello World")

**talk about what we are going to do**

In [ ]:
# This may need to happen first, to get stopwords downloaded to all learners' home folder?
# import nltk
# nltk.download('stopwords')

In [6]:
# import stuff and run it
from scipy import stats
import pandas
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

get the data and roadmap where we want to go. Maybe whiteboard the entire process?

In [1]:
# download data and do a little reality check
# Want a paper that includes volumes before, during and after WWI & flu; Phoenix Tribune?
title_1 = "phoenix-tribune"
year = "1919"
month = "05"
day = "17"
filename = "data/" + title_1 + "/volumes/" + year + month + day + ".txt"
print(filename)

data/phoenix-tribune/volumes/19190517.txt


In [3]:
# Open the file so we can read the text from it
file = open(filename, "r")
# Read the file and store in variable issue_text
issue_text = file.read()
# Close the file after reading in the text
file.close()
# Print the first 200 characters of the text
print(issue_text[0:200])

An Advertisement in the Tribune is a Direct Personal Appeal to the Colored People
PHOENIX A TRIB ONE
- VOLUME 11. NO. 7
White Editor Champions Cause of Black Man
BRAVE CMED BOVS 10 FOOGHT
IN FRANCE MA


some word counting before removing stopwords

In [ ]:
# word count and point out "the"

removal of stopwords and maybe single characters?

In [ ]:
# load corpora, remove stopwords, and look at results of count

## Beyond counting
There is a lot more we can do more than just count words. For example, we can look for specific words and see how their frequency changes over time. Given the publication dates of the newspapers we are looking at and current events, we can look at the frequency of the words "flu" and "influenza". And see how this frequency is changing over time.

In [ ]:
# Search for "flu" and "influenza" in one volume of interest (middle of influenza pandemic)

Now get relative frequency for all volumes in a year

In [ ]:
# Loop over all volumes in a single year, calculating frequency of flu and influenza for each volume

Comparative analysis - two papers

In [ ]:
# Loop over all volumes of second title in a single year, calculating rel. freq. of flu & influenza
# Second title is Bisbee Daily Review?

T-test comparing the two papers